**Alumno:  Isaias Eleuterio Tenorio Retis**

**Consigna del desafío 1**

In [1]:
import random
import pandas as pd
import numpy as np
import re
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import GridSearchCV

from sklearn.metrics.pairwise import cosine_similarity

# Fijamos la semilla del generador de números aleatorios
random.seed(42)

**1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.**

Cargamos el dataset fetch_20newsgroups

In [2]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [3]:
# Accedemos a los textos de los documentos
documentos = dataset.data

In [4]:
# Accedemos a las etiquetas de clasificación
etiquetas = dataset.target
etiquetas

array([10,  3, 17, ...,  3,  1,  7])

In [5]:
# Accedemos a los nombres de las categorías
categorias = dataset.target_names
categorias

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
# Determinamos el número total de documentos
total_documentos = len(dataset.data)
total_documentos

18846

In [7]:
# Visualizamos las 5 primeras y las 5 últimas etiquetas y sus correspondientes categorías
print("Primeras 5 etiquetas:")
for i in range(5):
    print(f"Documento {i}: Etiqueta numérica = {etiquetas[i]}, Categoría = {categorias[etiquetas[i]]}")

print("\nÚltimas 5 etiquetas:")
for i in range(total_documentos-5, total_documentos):
    print(f"Documento {i}: Etiqueta numérica = {etiquetas[i]}, Categoría = {categorias[etiquetas[i]]}")

Primeras 5 etiquetas:
Documento 0: Etiqueta numérica = 10, Categoría = rec.sport.hockey
Documento 1: Etiqueta numérica = 3, Categoría = comp.sys.ibm.pc.hardware
Documento 2: Etiqueta numérica = 17, Categoría = talk.politics.mideast
Documento 3: Etiqueta numérica = 3, Categoría = comp.sys.ibm.pc.hardware
Documento 4: Etiqueta numérica = 4, Categoría = comp.sys.mac.hardware

Últimas 5 etiquetas:
Documento 18841: Etiqueta numérica = 13, Categoría = sci.med
Documento 18842: Etiqueta numérica = 12, Categoría = sci.electronics
Documento 18843: Etiqueta numérica = 3, Categoría = comp.sys.ibm.pc.hardware
Documento 18844: Etiqueta numérica = 1, Categoría = comp.graphics
Documento 18845: Etiqueta numérica = 7, Categoría = rec.autos


In [8]:
# Creamos una función para limpiar texto
def limpiar_texto(texto):
    # Con esto eliminamos caracteres no deseados
    texto = re.sub(r'[^a-zA-Z0-9\s.,!?\'"]+', ' ', texto)
    # Con esto reemplazamos espacios múltiples con un solo espacio y quitamos espacios al principio y final
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

In [9]:
# Limpiamos documentos
documentos = [limpiar_texto(doc) for doc in documentos]

Vectorizamos los documentos usando TF-IDF

In [10]:
vectorizamos = TfidfVectorizer(stop_words='english')
tfidf_matriz = vectorizamos.fit_transform(documentos)
tfidf_matriz

<18846x129734 sparse matrix of type '<class 'numpy.float64'>'
	with 1225638 stored elements in Compressed Sparse Row format>

Selección aleatoria de 5 documentos

In [11]:
indices_aleatorios = random.sample(range(len(documentos)), 5)

Ahora medimos la similaridad entre los documentos seleccionados y el resto. Y por último encontramos los 5 documentos más similares para cada uno de los seleccionados.

In [12]:
# Medimos la similitud de cada documento seleccionado con el resto
resultados_similitud = {}
for idx in indices_aleatorios:
    documento_tfidf = tfidf_matriz[idx]
    similitudes = cosine_similarity(documento_tfidf, tfidf_matriz).flatten()

    # Obtener los índices de los documentos más similares
    indices_similares = similitudes.argsort()[-6:-1][::-1]  # 5 documentos más similares (sin incluir el propio documento)
    resultados_similitud[idx] = indices_similares

# Mostramos resultados
for idx, similares in resultados_similitud.items():
    print(f"\nDocumento {idx} (Categoría: {categorias[etiquetas[idx]]})")
    for sim_idx in similares:
        if sim_idx < len(documentos):  # Verificar que el índice está dentro del rango
            print(f"  - Documento {sim_idx} (Categoría: {categorias[etiquetas[sim_idx]]}), Similaridad: {cosine_similarity(tfidf_matriz[idx], tfidf_matriz[sim_idx])[0, 0]:.4f}")
            print(f"    Texto: {documentos[sim_idx][:500]} ...")  # Mostrar los primeros 500 caracteres del documento


Documento 3648 (Categoría: sci.crypt)
  - Documento 9850 (Categoría: sci.crypt), Similaridad: 0.2632
    Texto: Read it again yourself, then re apply the admonition you gave to the previous poster to yourself, as well. The first clause is not a condition, it is a reason for explicitly supporting the right WHICH EXISTS, MILITIA OR NOT, that the people have a right to keep and bear arms. This is NOT a right granted by the Constitution, it is a right presumed to exist by default. The Constitution mentioning a right is to prevent the government from removing that right by stating very clearly the government s ...
  - Documento 5229 (Categoría: talk.politics.guns), Similaridad: 0.2432
    Texto: It appears it is time that this article originally posted by Larry Cipriani last year, and which I saved gets posted again. It offers as good an analysis of the meaning of the Second Amendment, especially regarding the militia clause, as I have seen. I have not seen any rebuttles with similar bone 

**Conclusión**

En general, la similaridad entre los documentos es coherente con su contenido y categoría en la mayoria de los casos. Los documentos similares suelen pertenecer a la misma categoría y tratar temas relacionados. Sin embargo, hay excepciones donde la similaridad no es coherente con la categoria o contenido

**2.Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.**

Preparación del conjunto de datos

In [13]:
# Cargamos el conjunto de datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Dividimos en características y etiquetas
X_train, X_test, y_train, y_test = newsgroups_train.data, newsgroups_test.data, newsgroups_train.target, newsgroups_test.target

# Vectorización TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [14]:
X_train_tfidf

<11314x39115 sparse matrix of type '<class 'numpy.float64'>'
	with 693602 stored elements in Compressed Sparse Row format>

In [15]:
X_test_tfidf

<7532x39115 sparse matrix of type '<class 'numpy.float64'>'
	with 422138 stored elements in Compressed Sparse Row format>

Entrenamos y evaluamos los modelos de Naive Bayes

In [16]:
# Modelo Naïve Bayes Multinomial
mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train)
y_pred_mnb = mnb.predict(X_test_tfidf)

# Modelo ComplementNB
cnb = ComplementNB()
cnb.fit(X_train_tfidf, y_train)
y_pred_cnb = cnb.predict(X_test_tfidf)

# Evaluación con F1-score macro
f1_mnb = f1_score(y_test, y_pred_mnb, average='macro')
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')

print(f"F1-score macro para MultinomialNB: {f1_mnb:.4f}")
print(f"F1-score macro para ComplementNB: {f1_cnb:.4f}")

# Informe de clasificación detallado
print("\nReporte de clasificación para MultinomialNB:\n", classification_report(y_test, y_pred_mnb))
print("\nReporte de clasificación para ComplementNB:\n", classification_report(y_test, y_pred_cnb))


F1-score macro para MultinomialNB: 0.6512
F1-score macro para ComplementNB: 0.6943

Reporte de clasificación para MultinomialNB:
               precision    recall  f1-score   support

           0       0.74      0.20      0.32       319
           1       0.65      0.69      0.67       389
           2       0.66      0.60      0.63       394
           3       0.61      0.74      0.67       392
           4       0.78      0.68      0.73       385
           5       0.81      0.76      0.79       395
           6       0.78      0.78      0.78       390
           7       0.81      0.73      0.77       396
           8       0.85      0.75      0.80       398
           9       0.91      0.80      0.85       397
          10       0.57      0.93      0.71       399
          11       0.64      0.79      0.70       396
          12       0.71      0.53      0.61       393
          13       0.88      0.76      0.81       396
          14       0.76      0.74      0.75       394
     

Los modelos multinominalNB y complementNB mostraron un desempeño similar en la clasificación de textos. La precisión global fue del 68% y del 71% respectivamente. Ambos modelos tienen fortalezas y debilidades en diferentes clases. En general el modelo complementNB mostró un desempeño ligeramente mejor que multinomialNB. Ambos modelos pueden ser considerados buenos para tareas de clasificación de texto.

Ajuste de parámetros

In [17]:
# Parámetros a ajustar para el modelo MultinomialNB
param_grid_mnb = {
    'alpha': [0.1, 0.5, 1, 2, 5],
    'fit_prior': [True, False]
}

# Parámetros a ajustar para el modelo ComplementNB
param_grid_cnb = {
    'alpha': [0.1, 0.5, 1, 2, 5],
    'norm': [True, False]
}

# Creamos objetos GridSearchCV para cada modelo
grid_search_mnb = GridSearchCV(MultinomialNB(), param_grid_mnb, cv=5, scoring='f1_macro')
grid_search_cnb = GridSearchCV(ComplementNB(), param_grid_cnb, cv=5, scoring='f1_macro')

# Ajustamos los hiperparámetros utilizando GridSearchCV
grid_search_mnb.fit(X_train_tfidf, y_train)
grid_search_cnb.fit(X_train_tfidf, y_train)

# Imprimimos los mejores parámetros y el mejor puntaje F1
print("Mejores parámetros para MultinomialNB:", grid_search_mnb.best_params_)
print("Mejor puntaje F1 para MultinomialNB:", grid_search_mnb.best_score_)

print("Mejores parámetros para ComplementNB:", grid_search_cnb.best_params_)
print("Mejor puntaje F1 para ComplementNB:", grid_search_cnb.best_score_)

Mejores parámetros para MultinomialNB: {'alpha': 0.1, 'fit_prior': False}
Mejor puntaje F1 para MultinomialNB: 0.7584425712558112
Mejores parámetros para ComplementNB: {'alpha': 0.1, 'norm': True}
Mejor puntaje F1 para ComplementNB: 0.7659438101895686


**Conclusión**

Se encontraron los mejores parámetros para los modelos multinomialNB y complementNB. el modelo complementNB supera al modelo multinomialNB en términos de puntaje F1. Ambos modelos tienen un buen desempeño con un valor pequeño de alpha. la optimización de hiperparámetos mejora la precisión de la clasificación de textos. El modelo complementNB es la mejor opción para clasificación de texto en este caso.

**3.-Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [18]:
# Transponemos la matriz documento-término para obtener la matriz término-documento
X_tfidf_transposed = X_train_tfidf.T

In [19]:
# Obtenemos los términos (palabras) en el vocabulario
terms = vectorizamos.get_feature_names_out()

In [20]:
#  Seleccionamos manualmente las palabras relevantes
selected_words = ['computer', 'cars', 'big', 'conference', 'algorithm']

In [ ]:
# Encontramos los índices de las palabras seleccionadas en la matriz término-documento
selected_word_indices = []
for word in selected_words:
    if word in terms:
        index = np.where(terms == word)[0][0]
        selected_word_indices.append(index)
    else:
        print(f"La palabra '{word}' no se encontró en el vocabulario")

# Calculamos la similitud del coseno entre los términos, si hay palabras válidas
if selected_word_indices:
    similarity_matrix = cosine_similarity(X_tfidf_transposed)

    #  Para cada palabra seleccionada, encontrar las 5 palabras más similares
    for i, word_index in enumerate(selected_word_indices):
        if word_index < similarity_matrix.shape[0]:  # Verificamos que el índice esté en el rango válido
            word_similarities = similarity_matrix[word_index]
            # Encontramos los índices de las 5 palabras más similares (excluyendo la misma palabra)
            similar_word_indices = word_similarities.argsort()[-6:-1][::-1]  # Excluye la palabra misma
            similar_words = [terms[idx] for idx in similar_word_indices if idx < len(terms)]

            print(f"Palabra: {selected_words[i]}")
            print(f"5 palabras más similares: {similar_words}\n")
        else:
            print(f"Índice de palabra '{selected_words[i]}' fuera de rango")
else:
    print("No se encontraron palabras seleccionadas en el vocabulario.")